## リポジトリをクローン

In [18]:
import os
import subprocess
import sys

# --- 設定項目 ---

# 1. クローンしたいリポジトリのURLリストが書かれたファイル
url_file_path = 'url_list.txt'

# 2. クローンしたリポジトリを保存するディレクトリ名
clone_to_directory = 'cloned_repositories'


# --- 処理の開始 ---
print(f"--- Starting: Cloning repositories into '{clone_to_directory}' ---")

# 保存先ディレクトリが存在しない場合は作成
os.makedirs(clone_to_directory, exist_ok=True)
print("-" * 50)

# URLリストを読み込む
try:
    with open(url_file_path, 'r') as file:
        # 空行を除外してリスト化
        urls = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print(f"❌ Error: '{url_file_path}' was not found.")
    print("Please make sure the file exists in the same directory as the notebook.")
    urls = [] # エラーがあった場合はリストを空にする

# 各URLに対してループ処理
for repo_url in urls:
    # ★★★ ここからが修正箇所 ★★★

    # 1. URLからリポジトリ名を取得 (例: 'requests')
    repo_name = repo_url.split('/')[-1].replace('.git', '')
    
    # 2. クローン先に作られるであろうディレクトリのフルパスを構築
    repo_path = os.path.join(clone_to_directory, repo_name)
    
    # 3. ディレクトリが既に存在するか確認
    if os.path.isdir(repo_path):
        print(f"🟢 Skipping: {repo_name} (Directory already exists)")
        print("-" * 50)
        continue  # 存在する場合は、このURLの処理をスキップして次のループに進む

    # ★★★ 修正箇所ここまで ★★★

    print(f"Cloning: {repo_url}")
    try:
        # Popenでプロセスを開始し、出力をパイプで受け取る
        # --progressフラグで進捗表示を強制
        process = subprocess.Popen(
            ['git', 'clone', '--progress', repo_url],
            cwd=clone_to_directory,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE, # 進捗はこちらに出力される
            text=True,
            encoding='utf-8',
            errors='replace' # エンコーディングエラーを置換文字で回避
        )

        # 標準エラー(stderr)をリアルタイムで読み込んで進捗を表示
        while process.poll() is None:
            line = process.stderr.readline()
            if line:
                # 行末の改行を削除し、\rでカーソルを先頭に戻して上書き表示
                print(f"   {line.strip()}", end='\r')
        
        # プロセス完了後、進捗表示の行をクリアする
        print(" " * 80, end="\r")

        # プロセスの終了コードを確認
        if process.returncode == 0:
            print("✅ Clone successful.")
        else:
            # エラーが発生した場合、残りのエラー出力を取得して表示
            stdout_err, stderr_err = process.communicate()
            error_message = stderr_err if stderr_err else stdout_err
            print(f"❌ Error cloning. Reason: {error_message.strip()}")

    except FileNotFoundError:
        print("❌ Error: 'git' command not found. Please install Git and ensure it is in your system's PATH.")
        break # Gitがない場合は処理を中断
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("-" * 50)

print("Clone process finished.")

--- Starting: Cloning repositories into 'cloned_repositories' ---
--------------------------------------------------
🟢 Skipping: sbom-python-test (Directory already exists)
--------------------------------------------------
🟢 Skipping: sbom-go-test (Directory already exists)
--------------------------------------------------
🟢 Skipping: Python (Directory already exists)
--------------------------------------------------
🟢 Skipping: python-mini-projects (Directory already exists)
--------------------------------------------------
Clone process finished.


## sbom-toolのsbomを生成

In [19]:
import os
import subprocess
import re
import shutil

# --- 設定項目 ---
clone_to_directory = 'cloned_repositories'
sbom_output_directory = 'generated_sboms'

# --- 処理の開始 ---
print(f"--- Starting: Generating and moving SBOMs ---")
print("-" * 50)

original_path = os.getcwd()
os.makedirs(sbom_output_directory, exist_ok=True)

try:
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
         print("No repositories found to run commands on.")

    for repo_name in repo_dirs:
        final_repo_dir = os.path.join(original_path, sbom_output_directory, repo_name)
        final_manifest_path = os.path.join(final_repo_dir, '_manifest')

        if os.path.isdir(final_manifest_path):
            print(f"🟢 Skipping: {repo_name} (SBOM manifest already exists)")
            print("-" * 50)
            continue

        repo_path = os.path.join(clone_to_directory, repo_name)
        print(f"▶️  Entering: {repo_name}")
        
        try:
            os.chdir(repo_path)
            
            # --- パラメータ取得 ---
            package_name = repo_name
            git_version_result = subprocess.run(['git', 'rev-parse', 'HEAD'], capture_output=True, text=True, check=True)
            package_version = git_version_result.stdout.strip()
            
            package_supplier = "Unknown"
            with open('.git/config', 'r') as config_file:
                match = re.search(r'url\s*=\s*https?://github\.com/([^/]+)/', config_file.read())
                if match:
                    package_supplier = match.group(1)
            
            print(f"   Package Name: {package_name}")
            print(f"   Package Version: {package_version[:12]}...")
            print(f"   Package Supplier: {package_supplier}")
            
            # --- sbom-tool実行 ---
            command = [
                'sbom-tool', 'generate', '-bc', '.', '-b', '.',
                '-pn', package_name, '-pv', package_version, '-ps', package_supplier
            ]
            print(f"   Executing: {' '.join(command)}")
            subprocess.run(command, check=True, capture_output=True, text=True)
            print("✅ SBOM generation successful.")

            # --- 移動処理 ---
            source_manifest_path = '_manifest'
            
            if os.path.isdir(source_manifest_path):
                os.makedirs(final_repo_dir, exist_ok=True)
                print(f"   Moving '{source_manifest_path}' into: {final_repo_dir}")
                shutil.move(source_manifest_path, final_repo_dir)
                print("✅ Move successful.")
            else:
                print(f"⚠️ Warning: Could not find generated manifest directory at '{source_manifest_path}'")

        except FileNotFoundError:
            print(f"❌ Error: The command 'sbom-tool' or 'git' was not found.")
            break
        except subprocess.CalledProcessError as e:
            # ★★★ ここからが修正箇所 ★★★
            print(f"❌ Error executing command in {repo_name}.")
            # stdoutとstderrの両方を出力して、より多くの情報を得る
            if e.stdout:
                print(f"   [stdout]:\n{e.stdout.strip()}")
            if e.stderr:
                print(f"   [stderr]:\n{e.stderr.strip()}")
            # ★★★ 修正箇所ここまで ★★★
        finally:
            os.chdir(original_path)
            print("-" * 50)

except FileNotFoundError:
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")

print("All processes finished.")

--- Starting: Generating and moving SBOMs ---
--------------------------------------------------
🟢 Skipping: sbom-python-test (SBOM manifest already exists)
--------------------------------------------------
🟢 Skipping: Python (SBOM manifest already exists)
--------------------------------------------------
🟢 Skipping: python-mini-projects (SBOM manifest already exists)
--------------------------------------------------
🟢 Skipping: sbom-go-test (SBOM manifest already exists)
--------------------------------------------------
All processes finished.
